In [1]:
from models.auxiliary import Auxiliary
from models.scenario import Scenario
from models.scenario_manager import ScenarioManager
from models.calibrator import Calibrator
from models.visualization import Visualization
import pandas as pd


auxiliaries = [
    Auxiliary("rabbit_reproduction_rate", [0.1] * 1000),  # Constant value over time 
    Auxiliary(
        "hunting_rate", [0.02, 0.015, 0.01, 0.005] * 250 
    ),  # Pattern repeated over time
    Auxiliary(
        "fox_reproduction_rate", [0.01]
    ),  # Constant, using a list for consistency
    Auxiliary("fox_death_rate", 0.1),  # Constant, using a float
]

# Initial values for the stocks
initial_values = {"rabbits": 100, "foxes": 20}

# Rates and rate functions for the flows
rates = {
    "rabbit_birth": {
        "rate_function": lambda rabbits, rabbit_reproduction_rate: rabbits
        * rabbit_reproduction_rate,
        "source": None,
        "destination": "rabbits",
    },
    "rabbit_death": {
        "rate_function": lambda rabbits, foxes, hunting_rate: rabbits
        * foxes
        * hunting_rate,
        "source": "rabbits",
        "destination": None,
    },
    "fox_birth": {
        "rate_function": lambda rabbits, foxes, fox_reproduction_rate: rabbits
        * foxes
        * fox_reproduction_rate,
        "source": None,
        "destination": "foxes",
    },
    "fox_death": {
        "rate_function": lambda foxes, fox_death_rate: foxes * fox_death_rate,
        "source": "foxes",
        "destination": None,
    },
}
scenario1 = Scenario("Base Scenario", {"rabbits": 30, "foxes": 20}, rates, auxiliaries)


In [2]:
target_data = pd.read_excel('calibration_data_test.xlsx')
df = target_data[['Predator','Prey']]
df.columns = ['foxes','rabbits']
df

,foxes,rabbits
0,19.58,30.09
1,19.60,45.15
2,19.61,49.15
3,11.99,39.52
4,28.04,21.23
...,...,...
86,19.52,8.31
87,82.11,16.01
88,89.76,24.82
89,81.66,29.70


In [3]:
# Création de l'objet Calibrator et exécution de la calibration
calibrator = Calibrator(scenario1)
optimized_params = calibrator.calibrate(df, method="least_squares")

/tmp/ipykernel_152076/2674402800.py:32: RuntimeWarning: overflow encountered in scalar multiply
  "rate_function": lambda rabbits, foxes, hunting_rate: rabbits
/tmp/ipykernel_152076/2674402800.py:39: RuntimeWarning: overflow encountered in scalar multiply
  "rate_function": lambda rabbits, foxes, fox_reproduction_rate: rabbits


ValueError: Stock value became non-finite

In [ ]:
optimized_params

array([0.1 , 0.02, 0.01, 0.1 ])